## Obtain Credentials (not everytime)

Google Earth Engine (GEE) periodically requests users to renew their credentials by calling the 'ee.Authenticate()' function. Upon calling this function, a web page will open where you can sign in with your Google account. This step allows GEE to authenticate your identity and associate your API requests with your Google account. After the authentication process is successfully completed, a persistent token will be stored on your local machine (valid for a few days). 

In [2]:
import ee
ee.Authenticate()

Enter verification code:  4/1AdLIrYfSwPQNYJdR4Vjazqge05uUCjqB5UJv5PdlNoA5SAvIzKo5hG7Ccxc



Successfully saved authorization token.


## Initialize with Regular Google Account

This is normally the start point to execute the LEAF Production Tool. By calling the 'ee.Initialize()' function, the saved credentials will be automatically retrieved, allowing you to make requests through the GEE's Python API.

In [1]:
import ee
ee.Initialize()

#import geemap
#import Image
import LEAFNets as LFNs

## Define Input Parameters

To run the LEAF production tool, a set of parameters must be supplied. To streamline the provision of these parameters, a Python dictionary serves as a container to hold 13 “key:value” pairs, with the last three being optional. Each parameter is detailed as follows:

(1) Value for 'sensor' key: a string denoting the sensor type and data unit. Currently, valid values include 'L8_SR', 'S2_SR', 'L8_TOA' and 'S2_TOA', which denote Landsat 8/9 and Sentinel-2 surface reflectance and TOA reflectance data, respectively. 

(2) Value for 'year' key: a 4-digits integer representing the acquisition year (e.g., 2020) of the daily images used in the production.

(3) Value for 'months' key: a list of integers specifying the months of the year (e.g., [6, 7] for June and July). With a list of month numbers, the LEAF tool can generate multiple monthly products in a single execution. To create products for a peak season (June 15 to September 15) of a year (defined by the value of the 'year' key), simply include a negative integer in the list.

(4) Value for 'prod_names' key: a list of biophysical parameter name strings, which can be a subset or all of the elements in ['LAI', 'fCOVER', 'Albedo', 'fAPAR', 'QC', 'date', 'partition'].

(5) Value for 'tile_names' key: a set of strings representing the names of grid tiles, each covering a 900km x 900km area. By providing a list of tile names, the LEAF Tool can produce biophysical parameter products for multiple tiles in a single execution. To generate biophysical products for a customized region, include an additional "key:value" pair in this parameter dictionary (see the value specification for the 'custom_region' key. 

(6) Value for 'spatial_scale' key: an integer defining the spatial resolution (in meters) for exporting the resultant products (e.g., 30 and 20 for Landsat and Sentinel-2, respectively).

(7) Value for 'out_location' key: a string specifying the destination for exporting the resultant products. Valid values are 'drive' for Google Drive (GD) or 'storage' for Google Cloud Storage (GCS).

(8) Value for 'GCS_bucket' key: a string indicating a bucket name on GCS. This parameter is necessary only when the exporting destination is GCS ('storage' for 'out_location' key). Note that the specified bucket must exist on GCS before exporting. 

(9) Value for 'out_folder' key: the folder name on GD or GCS for exporting products. If you prefer not to export all the products to the same directory, leave an empty string for this key. The Tool will then automatically create separate folders for the products of different tiles according to tile name and acquisition year.

(10) Value for 'custom_region' key: an "ee.Geometry" object created using the "ee.Geometry.Polygon()" function, which takes a list of Latitude and Longitude coordinates as inputs. Include this 'key:value' pair only when a customized region has to be defined. Otherwise, DO NOT include this 'key:value' pair in the parameter dictionary, as it will overwrite the values for the 'tile_names' key. 

(11) Value for 'start_date' key: a string (e.g., '2022-06-15') to specify the start date of a customized timeframe. 

(12) Value for 'end_date' key: a string (e.g., '2022-09-15') to specify the end date of a custmoized timeframe. Please note that the strings for 'start_date' and 'end_date' keys should be omitted from this parameter dictionary unless a user-defined timeframe needs to be specified.

(13) Value for 'projection' key: a string specifying a customized projection. The default projection used in the LEAF Tool is 'EPSG:3979'.

Among the 12 input parameters, 3 keys ('months', 'prod_names' and 'tile_names') require list inputs. By combining these lists differently, you can handle various scenarios for producing vegetation biophysical parameter maps. For instance, to generate monthly (e.g., July and August) biophysical parameter maps for multiple tiles (e.g., 'tile41', 'tile42' and 'tile43'), two lists, [7, 8] and ['tile41', 'tile42', 'tile43'], should be specified for 'months' and 'tile_names' keys, respectively.    

In [2]:
ottawa_region = ee.Geometry.Polygon([[-76.12016546887865,45.183832177265906], [-75.38339483899584,45.170763450281996],
                                     [-75.39026129407397,45.5639242833682], [-76.10505926770678,45.56776998764525], 
                                     [-76.12016546887865,45.183832177265906]])

user_region = ee.Geometry.Polygon([[0.9637900778919472,50.91403368831729],[0.9702334792675978,50.91699007337573], [1.373015219949208,51.143035232142026],
                          [1.392425687228554,51.161210528531235], [1.3993416800246405,51.17487321017443], [1.7590038910436083,52.48232628523823], 
                            [1.7397985726778475,52.632339348548456],[-0.7617973294684357,60.818076114961116],[-0.7720487663815042,60.82957167065116],
                          [-0.84006341916535,60.84228905763035],[-0.8825987706179472,60.84579837326019],[-0.8989190990526906,60.84100929771918],
                            [-6.151035538784849,59.10082279852729], [-8.649648277561651,57.8312632023955], [-8.65067837292272,57.82940821160297],
                           [-8.181906429548674,54.466265077464875], [-6.358374907753275,49.891254089262475], [-6.348185796971474,49.88672370082926],
                         [-6.345300748028998,49.88588533845501], [-5.207257982555416,49.961039056211824], [-5.202317312264887,49.96157416319619],
                            [0.23758982626651468,50.73736405209513], [0.25584985355543854,50.74020452310862],[0.9637900778919472,50.91403368831729]])


In [3]:
params = {
    'sensor': 'S2_SR',               # A string indicating sensor type and data unit (e.g., 'S2_SR' or 'L8_SR')    
    'year': 2023,                    # An integer representing image acquisition year
    'months': [7],        # A set of integers representing months of the year (a negative value means a peak season from Jun.15 to Sept.15)       
    'prod_names': ['LAI','fAPAR','fCOVER','Albedo', 'date', 'partition'],   # A subset or all of the elements in ['LAI','fAPAR','fCOVER','Albedo', 'date', 'partition']
    'tile_names': ['tile55'],        # A list of tile name strings as per the CCRS' tile griding system
    'spatial_scale': 20,             # The spatial resultion (in meter) of the exported products
    'out_location': 'drive',         # The destination of exporting the products ('drive' or 'storage')     
    'GCS_bucket': 'LEAF_production', # An unique bucket name on Google Cloud Storage (must have been created beforehand)
    'out_folder': 'S2_leaf_2023_EPSG4326',    # The folder name intended for exporting products on either Google Drive or Google Cloud Storage
    'custom_region': ottawa_region, # A given user-defined region. Only include this 'key:value' pair as necessary
    'start_date': '2022-06-15',     # A string to specify the start date of a customized compositing period
    'end_date': '2022-09-15',       # A string to specify the end date of a customized compositing period
    'projection': 'EPSG:4326'       # A string representing a customized projection. Default projection = 'EPSG:3979'
    #'scene_ID': 'LC08_016029_20230814'
}

## Generate and Export Biophysical Products

In [4]:
LFNs.LEAF_production(params)

<LEAF_production> All input parameters =  {'sensor': 'S2_SR', 'unit': 2, 'year': '2022', 'nbYears': -1, 'months': [7], 'tile_names': ['tile55'], 'prod_names': ['LAI', 'fAPAR', 'fCOVER', 'Albedo', 'date', 'partition'], 'out_location': 'drive', 'spatial_scale': 20, 'GCS_bucket': 'LEAF_production', 'out_folder': 'S2_leaf_2023_EPSG4326', 'export_style': 'separate', 'start_date': '2022-06-15', 'end_date': '2022-09-15', 'scene_ID': '', 'projection': 'EPSG:4326', 'CloudScore': False, 'current_month': -1, 'current_tile': '', 'time_str': '2022-06-15_2022-09-15', 'region_str': 'custom_region', 'custom_region': ee.Geometry({
  "functionInvocationValue": {
    "functionName": "GeometryConstructors.Polygon",
    "arguments": {
      "coordinates": {
        "constantValue": [
          [
            [
              -76.12016546887865,
              45.183832177265906
            ],
            [
              -75.38339483899584,
              45.170763450281996
            ],
            [
        

[None, None, None, None, None, None, None]

## Check the Status of Exporting Tasks (optional)

Depending on whether you are using a regular Google account or a Google Service account to operate Google Earth Engine, there are two methods available to monitor the status of exporting tasks. If you're using a regular Google account, you can check the status by either accessing the "Tasks" tab located in the right panel of your Earth Engine Code Editor or executing the code provided in the subsequent cell. However, if you're using a Google Service account, the only way to monitor the status of exporting tasks is by running the code in the following cell.

The function "manage_tasks" requires two input parameters. The first parameter is a string and can take either 'status' or 'cancel' as its value. 'status' is used to check the status of exporting tasks, while 'cancel' is used to cancel all the exporting tasks. The second parameter is a string used to filter task names. For instance, providing '2019' will retrieve tasks with names that include '2019'. 

In [4]:
Image.manage_tasks('cancel', '2020')

tile55_2020_Jul_QC_200m has been cancelled.
tile55_2020_Jul_Albedo_200m has been cancelled.
tile55_2020_Jul_fAPAR_200m has been cancelled.
tile55_2020_Jul_fCOVER_200m has been cancelled.
tile55_2020_Jul_LAI_200m has been cancelled.
tile55_2020_Jul_QC_200m has been cancelled.
tile55_2020_Jul_Albedo_200m has been cancelled.
tile55_2020_Jul_fAPAR_200m has been cancelled.
tile55_2020_Jul_fCOVER_200m has been cancelled.
tile55_2020_Jul_LAI_200m has been cancelled.
tile55_2020_Jul_QC_200m has been cancelled.
tile55_2020_Jul_Albedo_200m has been cancelled.
tile55_2020_Jul_fAPAR_200m has been cancelled.
tile55_2020_Jul_fCOVER_200m has been cancelled.
tile55_2020_Jul_LAI_200m has been cancelled.
tile55_2020_Jul_QC_200m has been cancelled.
tile55_2020_Jul_Albedo_200m has been cancelled.
tile55_2020_Jul_fAPAR_200m has been cancelled.
tile55_2020_Jul_fCOVER_200m has been cancelled.
tile55_2020_Jul_LAI_200m has been cancelled.
tile55_2020_Jul_QC_200m has been cancelled.
tile55_2020_Jul_Albedo_200m 